# Trabalho BI - Tópicos Avançados

## A base do trabalho contém todas as reclamações fundamentadas que foram audiência no ano de 2017.

### O notebook cria as seguintes WordClouds:
###    1 - Proporção de reclamações por Estado, utilizando a coluna 'estado'.
###    2 - As 20 empresas mais reclamadas por Estado, utilizando as colunas 'estado' e 'empresa'.
####           - Criação da WordCloud utilizando Expressões Regulares para Pré-processamento.
####           - Criação da WordCloud utilizando Expressões Regulares, Stopwords, Remoção de Acentuação e Tokenização para Pré-processamento.

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.tokenize import word_tokenize
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Carregamento da base
data = pd.read_csv(r'C:\Users\rfsantos\OneDrive - Mongeral Aegon\2020\Modulos\BI\NLP\Trabalho\Arquivos fonte finais\base reclamacoes Procon.csv', sep = ';', encoding ='windows-1252', keep_default_na = True)

In [ ]:
# Existem valores nulos (por coluna)?
data.isna().sum()

In [ ]:
# Remoção de linhas com valores nulos
data = data.dropna(subset=['empresa','problema']).astype(str)

## 1 - WordCloud com a proporção de reclamações por estado no Procon em 2017

In [ ]:
# Criação de dataframe com a coluna estado
data_estado = pd.DataFrame({'estado':data.estado})

In [ ]:
# Criação de uma string única com todas as ocorrências dos estados
string_estados = " ".join(text for text in data_estado.estado)

In [ ]:
# Definição de imagem para formato da Wordcloud
brazil_mask = np.array(Image.open(r'C:\Users\rfsantos\OneDrive - Mongeral Aegon\2020\Modulos\BI\NLP\Trabalho\Arquivos fonte finais\brazil-map.png'))
img=mpimg.imread(r'C:\Users\rfsantos\OneDrive - Mongeral Aegon\2020\Modulos\BI\NLP\Trabalho\Arquivos fonte finais\brazil-map.png')
imgplot = plt.imshow(img)
plt.axis("off");

In [ ]:
# WordCloud
cloud = WordCloud(background_color='black',width=1600, height=800,max_font_size=200,max_words=20, mask=brazil_mask, contour_width=1, contour_color='steelblue', collocations=False).generate(string_estados)

plt.figure(figsize = (20,10))
plt.imshow(cloud, interpolation='nearest')
plt.title('Proporção de Reclamações por Estado', fontsize=20)
plt.axis('off')
plt.savefig('estados-total.png')
plt.tight_layout(pad=0)

## 2 - WordCloud com as 20 empresas mais reclamadas no Procon em 2017 por Estado

### - Criação da WordCloud utilizando Expressões Regulares para Pré-processamento

In [ ]:
# Criação de dataframe com as colunas empresa e estado
data_estado_1 = pd.DataFrame({'estado':data.estado,'empresa':data.empresa.str.lower()})

In [ ]:
# Utilização de expressões regulares para limpeza do texto
dicionario_1 = {' s/a':'',' s.a':'',' ltda':'',' s/s':'',',':'',\
                     '\(':'','\)':'',"-":'','\.com':'','\.br':'',r"^\t":'',' epp':'',' lta':'','@':''}
data_estado_1 = data_estado_1.replace({'empresa': dicionario_1},regex=True)

In [ ]:
# Utilização de expressões regulares para limpeza do texto
dicionario_2 = {'_sa$':'','\.':'','/':' ',' ':'_'}
data_estado_1 = data_estado_1.replace({'empresa': dicionario_2},regex=True)

In [ ]:
# Criação da WordCloud

labels = np.unique(data_estado_1['estado'])
for label in labels:
    string_estado_1 = []
    phrases = data_estado_1[data_estado_1['estado'] == label]
    for phrase in phrases['empresa']:
        string_estado_1.append(phrase)      
    texto_estado_1 = pd.Series(string_estado_1).astype(str).str.cat(sep=' ').replace("'","")

    cloud1 = WordCloud(background_color='black',width=1600, height=800,max_font_size=200,max_words=20,collocations=False).generate(texto_estado_1)

    plt.figure(figsize = (20,10))
    plt.imshow(cloud1, interpolation='bilinear')
    plt.title('Empresas com mais reclamações em ' + str(label), fontsize=20)
    plt.axis('off')
    plt.tight_layout(pad=0)
    name = str(label)
    plt.savefig('estado-' + str(label) + ' regex.png')

### - Criação da WordCloud utilizando Expressões Regulares, Stopwords, Tokenização e Ngrams para Pré-processamento

In [ ]:
# Criação de dataframe com as colunas estado e empresa
data_estado_2 = pd.DataFrame({'estado':data.estado,'empresa':data.empresa.str.lower()})

In [ ]:
# Utilização de expressões regulares para limpeza do texto
dicionario_1 = {' s/a':'',' s.a':'',' s/s':'',',':'','\(':'','\)':'',"-":'','\.com':'','\.br':'',r"^\t":'','@':''}

data_estado_2 = data_estado_2.replace({'empresa': dicionario_1},regex=True)

In [ ]:
# Utilização de expressões regulares para limpeza do texto
dicionario_2 = {'\.':'','/':' '}
data_estado_2 = data_estado_2.replace({'empresa': dicionario_2},regex=True)

In [ ]:
# Expressões regulares - remoção de todas as palavras com números, pontuação e formatação de todas as palavras em minúsculas
alphanumeric = lambda x: re.sub(r"""\w*\d\w*""", ' ', x)
#punc = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)

data_estado_2.empresa = data_estado_2.empresa.map(alphanumeric) #.map(punc)

In [ ]:
# Remoção de acentos
data_estado_2['empresa'] = data_estado_2.empresa.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
# Tokenização da coluna empresa
data_estado_2['empresa'] = data_estado_2.empresa.map(word_tokenize)

In [ ]:
# Remoção de StopWords
stopword_list = nltk.corpus.stopwords.words('portuguese')
new_stopwords = ['d','co','kd','texto','object','dtype','i','c','outros','etc','sa','ltda','ltd','lta','epp']
stopword_list.extend(new_stopwords)

stop_lambda = lambda x: [y for y in x if y not in stopword_list]

data_estado_2['empresa'] = data_estado_2.empresa.map(stop_lambda)

In [ ]:
# Criação de lista com os Unigrams
#unigram_list = [] 
#for row in unigram.empresa:
#    unigram_list.extend(row)

In [ ]:
# Criação da WordCloud

## Criação de lista com os Ngrams
labels = np.unique(data_estado_2['estado'])
for label in labels:
    tokens_list = []
    rows = data_estado_2[data_estado_2['estado'] == label]
    for row in rows.empresa:
        tokens_list.append(row)
    ngrams_list = []
    for words in tokens_list:
        ngrams_list.append(' '.join(words))

## Criação de dicionário de frequência das strings
    words_freq = {} 
    for items in ngrams_list: 
        words_freq[items] = ngrams_list.count(items)

## Plotagem da WordCloud       
    cloud2 = WordCloud(background_color='black',width=1600,height=800,max_font_size=200,max_words=20).generate_from_frequencies(words_freq)

    plt.figure(figsize = (20,10))
    plt.imshow(cloud2, interpolation='bilinear')
    plt.title('Empresas com mais reclamações em ' + str(label), fontsize=40)
    plt.axis('off')
    plt.tight_layout(pad=0)
    name = str(label)
    plt.savefig('estado-' + str(label) + ' ngram.png')
    